In [2]:
import base64
import datetime
import json
import os
import random
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

pd.options.display.max_colwidth = 1000

# Load the Data

In [3]:

# days_since_last_manager_activity	days since manager's last listing creation

In [4]:
raw_data = pd.read_json('tmp/raw_data.json', convert_dates=['created'])
#d = pd.read_json('tmp/raw_data.json')
raw_data.index.name = 'id'
d = pd.DataFrame()
print len(d)

0


In [5]:
d.loc[:, 'price_per_bedrooms'] = raw_data['price'] / (0.1 + raw_data['bedrooms'])
d.loc[:, 'rooms_diff'] = raw_data['bedrooms'] - raw_data['bathrooms']
d.loc[:, 'rooms_sum'] = raw_data['bedrooms'] + raw_data['bathrooms']
d.loc[:, 'bathrooms_per_bedrooms'] = raw_data['bathrooms'] / (raw_data['bedrooms'] + 0.1)
d.head()

,price_per_bedrooms,rooms_diff,rooms_sum,bathrooms_per_bedrooms
id,,,,
0,2681.818182,0.0,2.0,0.909091
1,1357.142857,1.0,3.0,0.476190
10,967.741935,1.5,4.5,0.483871
100,3416.363636,0.0,2.0,0.909091
1000,1571.428571,1.0,3.0,0.476190


In [18]:
timediff = (raw_data.created.max() - raw_data.created.min()).components
max_diff_hour = timediff.days * 24 + timediff.hours
print max_diff_hour

2135


In [20]:
def sort_list(l):
    l.sort()
    return l

managers_dates = raw_data.groupby('manager_id')['created'].apply(list).map(sort_list).to_dict()


In [12]:
(pd.Timestamp('2016-04-24 01:24:19') - pd.Timestamp('2016-04-24 01:11:26')).components.hours

0

In [23]:
def time_since_managers_last_listing(row):
    listing_dates = managers_dates[row['manager_id']]
    created = row['created']
    idx = listing_dates.index(created)
    if idx == 0:
        return np.nan
    else:
        timediff = (listing_dates[idx] - listing_dates[idx - 1]).components
        return 24 * timediff.days + timediff.hours
d.loc[:, 'hours_since_managers_last_listing'] = raw_data[['manager_id', 'created']].apply(lambda row: time_since_managers_last_listing(row), axis=1)

In [24]:
d.dtypes

price_per_bedrooms                   float64
rooms_diff                           float64
rooms_sum                            float64
bathrooms_per_bedrooms               float64
hours_since_managers_last_listing    float64
dtype: object

In [25]:
d.head()

,price_per_bedrooms,rooms_diff,rooms_sum,bathrooms_per_bedrooms,hours_since_managers_last_listing
id,,,,,
0,2681.818182,0.0,2.0,0.909091,47.0
1,1357.142857,1.0,3.0,0.476190,52.0
10,967.741935,1.5,4.5,0.483871,0.0
100,3416.363636,0.0,2.0,0.909091,601.0
1000,1571.428571,1.0,3.0,0.476190,146.0


# Save the Data

In [26]:
d.to_csv('tmp/features_others.csv')